모델의 성능을 확인하기 위한 코드이다.

#**1. 구글 드라이브를 코랩에 연결한다.**
> 이는 추후 모델을 불러오고, 학습할 데이터를 불러오기 위해 필요한 과정이다. \
 따라서 이 코드를 실행하기 전에, 구글 드라이브에 모델과 토크나이저, 전처리된 데이터를 업로드 해야 한다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


#**2. 테스트를 위해 필요한 라이브러리를 불러온다.**

In [ ]:
!pip install transformers seqeval[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 22.9 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 182 kB 49.2 MB/s 
     |████████████████████████████████| 7.6 MB 47.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=1040895208200b201a2ee2a60899340ec5f040092e05ce6e11376e4d74386cc9
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizerFast, ElectraConfig, ElectraForTokenClassification

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


#**3. 테스트 데이터셋을 불러온다.**
> ***데이터의 경로 입력/수정 필수!***

In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/2022_lesik_workspace/lesik/data/new_no_pool_test.tsv', sep = '\t', keep_default_na=False)
df.head()

,text,label
0,"2. 씻은 우람버섯은 물기를 제거하고, 포크를 이용하여 찢어 준다.",O O CV_STATE CV_STATE CV_INGREDIENT CV_INGREDI...
1,"3. 포크로 찢은 우람버섯은 뜨거운 물에 소금을 넣고 30초 정도 데친 후, 찬물에...",O O O O CV_STATE CV_STATE CV_INGREDIENT CV_ING...
2,"4. 찬물에 씻은 우람버섯은 물기를 제거하고, 그릇에 담는다.",O O O O CV_STATE CV_STATE CV_INGREDIENT CV_ING...
3,"5. 냄비에 참기름, 다진 마늘을 넣고 센 불로 볶는다.",O O O O CV_SEASONING O CV_STATE CV_INGREDIENT ...
4,1. 늙은 호박은 채 썰어 준비한다,O O CV_INGREDIENT CV_INGREDIENT CV_INGREDIENT ...


태그는 학습된 모델과 동일해야 되므로 변경해서는 안된다.

In [ ]:
# Split labels based on whitespace and turn them into a list
arr_labels = set()
for lb in df.label:
    lb = lb.split()
    for ll in lb:
        if ll not in arr_labels:
            arr_labels.add(ll)

#말뭉치 데이터에 포함된 총 태그
unique_labels = {'OGG_EDUCATION', 'MT_ELEMENT', 'AFW_OTHER_PRODUCTS', 'MT_ROCK', 'TI_OTHERS', 'PS_NAME', 'CV_BUILDING_TYPE', 'AM_REPTILIA', 'OGG_FOOD', 'AF_MUSICAL_INSTRUMENT', 'AF_BUILDING', 'AFA_MUSIC', 'CV_SPORTS_INST', 'QT_ORDER', 'TM_COLOR', 'LCG_MOUNTAIN', 'QT_MAN_COUNT', 'PS_CHARACTER', 'AM_OTHERS', 'OGG_LIBRARY', 'TMM_DISEASE', 'OGG_MEDICINE', 'LCG_ISLAND', 'TI_MINUTE', 'MT_CHEMICAL', 'TM_CELL_TISSUE_ORGAN', 'QT_OTHERS', 'CV_TRIBE', 'QT_TEMPERATURE', 'PT_FLOWER', 'OGG_POLITICS', 'DT_WEEK', 'FD_ART', 'AM_AMPHIBIA', 'FD_MEDICINE', 'AF_CULTURAL_ASSET', 'AF_TRANSPORT', 'EV_SPORTS', 'LCG_CONTINENT', 'PT_TREE', 'TMI_SERVICE', 'AM_MAMMALIA', 'TM_SPORTS', 'CV_INGREDIENT', 'OGG_HOTEL', 'QT_PHONE', 'CV_LANGUAGE', 'CV_FUNDS', 'CV_CURRENCY', 'FD_OTHERS', 'LCG_RIVER', 'LCP_CAPITALCITY', 'LC_OTHERS', 'QT_SIZE', 'TM_CLIMATE', 'TM_SHAPE', 'CV_POLICY', 'EV_ACTIVITY', 'TR_ART', 'QT_ADDRESS', 'OGG_RELIGION', 'CV_POSITION', 'FD_HUMANITIES', 'CV_CULTURE', 'QT_SPORTS', 'QT_ALBUM', 'CV_ART', 'CV_FOOD', 'CV_LAW', 'OGG_MILITARY', 'DT_DAY', 'FD_SOCIAL_SCIENCE', 'LCP_PROVINCE', 'CV_CLOTHING', 'TI_HOUR', 'DT_DYNASTY', 'DT_SEASON', 'FD_SCIENCE', 'TMI_HW', 'OGG_SPORTS', 'TR_OTHERS', 'TM_DIRECTION', 'TMI_SITE', 'QT_LENGTH', 'MT_METAL', 'LCG_OCEAN', 'DT_OTHERS', 'LCP_COUNTY', 'TMIG_GENRE', 'OGG_ECONOMY', 'TMI_SW', 'CV_SPORTS_POSITION', 'AFA_DOCUMENT', 'PT_OTHERS', 'AFA_ART_CRAFT', 'EV_OTHERS', 'TMI_EMAIL', 'QT_PRICE', 'EV_FESTIVAL', 'TI_SECOND', 'CV_TAX', 'O', 'QT_VOLUME', 'AF_WEAPON', 'LCG_BAY', 'OGG_SCIENCE', 'PT_FRUIT', 'CV_OCCUPATION', 'QT_CHANNEL', 'OGG_ART', 'AM_INSECT', 'CV_FOOD_STYLE', 'QT_PERCENTAGE', 'OGG_LAW', 'TR_SCIENCE', 'CV_RELATION', 'AM_PART', 'QT_AGE', 'TMI_MODEL', 'AM_BIRD', 'OGG_OTHERS', 'CV_SPORTS', 'DT_YEAR', 'LCP_COUNTRY', 'AFA_VIDEO', 'DT_GEOAGE', 'TI_DURATION', 'AM_TYPE', 'CV_SEASONING', 'AM_FISH', 'CV_PRIZE', 'PS_PET', 'AFW_SERVICE_PRODUCTS', 'TMI_PROJECT', 'CV_DRINK', 'LC_SPACE', 'LCP_CITY', 'EV_WAR_REVOLUTION', 'AFA_PERFORMANCE', 'QT_SPEED', 'PT_GRASS', 'DT_MONTH', 'PT_PART', 'OGG_MEDIA', 'PT_TYPE', 'TMM_DRUG', 'AF_ROAD', 'DT_DURATION', 'TR_MEDICINE', 'TR_HUMANITIES'}
print(unique_labels)

# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

prob_tag_list=[]
for lb in arr_labels:
    if lb not in labels_to_ids:
        prob_tag_list.append("'"+lb+"'")
print(",".join(prob_tag_list))

#말뭉치에 포함되어 있지 않는 태그들 추가
labels_to_ids['CV_ACT'] = 150
ids_to_labels[150] = 'CV_ACT'

labels_to_ids['CV_STATE'] = 151
ids_to_labels[151] = 'CV_STATE'

print(ids_to_labels)
print(len(ids_to_labels))

{'TI_OTHERS', 'QT_PERCENTAGE', 'AM_REPTILIA', 'LC_OTHERS', 'QT_VOLUME', 'CV_ART', 'OGG_LIBRARY', 'FD_ART', 'OGG_LAW', 'TR_OTHERS', 'QT_AGE', 'OGG_MEDIA', 'EV_FESTIVAL', 'OGG_OTHERS', 'OGG_MILITARY', 'AF_BUILDING', 'OGG_MEDICINE', 'CV_LAW', 'TMI_EMAIL', 'LCP_COUNTY', 'TMM_DISEASE', 'CV_DRINK', 'CV_BUILDING_TYPE', 'OGG_RELIGION', 'TMI_SW', 'AF_WEAPON', 'PT_TYPE', 'CV_CULTURE', 'DT_GEOAGE', 'AM_INSECT', 'QT_SPEED', 'DT_OTHERS', 'CV_LANGUAGE', 'AM_OTHERS', 'TMI_HW', 'AFW_SERVICE_PRODUCTS', 'PS_NAME', 'CV_PRIZE', 'AM_PART', 'EV_WAR_REVOLUTION', 'TMM_DRUG', 'PT_TREE', 'TM_CLIMATE', 'QT_OTHERS', 'PS_CHARACTER', 'DT_YEAR', 'QT_LENGTH', 'OGG_SPORTS', 'PT_GRASS', 'DT_SEASON', 'CV_POLICY', 'LCG_CONTINENT', 'LCP_COUNTRY', 'AM_BIRD', 'OGG_SCIENCE', 'AF_MUSICAL_INSTRUMENT', 'CV_CLOTHING', 'CV_SEASONING', 'QT_ALBUM', 'TR_MEDICINE', 'QT_SIZE', 'OGG_EDUCATION', 'CV_FUNDS', 'MT_METAL', 'TI_DURATION', 'TMIG_GENRE', 'FD_SCIENCE', 'DT_WEEK', 'MT_ELEMENT', 'TI_HOUR', 'OGG_HOTEL', 'CV_SPORTS_INST', 'EV_OTHER

In [ ]:
# Let's take a look at how can we preprocess the text - Take first example
text = df['text'].values.tolist()
m_len = 0
for t in text:
    if m_len < len(t):
        m_len = len(t)
        
example = text[1]

print(example)
print(m_len)

3. 포크로 찢은 우람버섯은 뜨거운 물에 소금을 넣고 30초 정도 데친 후, 찬물에 씻어 준다.
172


#**4. 테스트를 진행 할 최종 모델과 토크나이저를 불러온다.**
> ***모델, 토크나이저, epoch 입력/수정 필수!***



In [ ]:
def load(epoch):
    model_directory = '/content/gdrive/MyDrive/2022_lesik_workspace/lesik/model/FIXED_FINAL_EPOCH_'+ str(epoch) #모델 경로
    model = ElectraForTokenClassification.from_pretrained(model_directory, num_labels=len(labels_to_ids))
    model.to(device)
    
    tokenizer_directory = '/content/gdrive/MyDrive/2022_lesik_workspace/lesik/tokenizer/FIXED_FINAL_EPOCH_' +str(epoch) #토크나이저 경로
    tokenizer = ElectraTokenizerFast.from_pretrained(tokenizer_directory)
    
    return model, tokenizer

argument로 불러오기를 원하는 epoch를 적는다.

In [ ]:
epoch = 72   #원하는 epoch 변경
model, tokenizer = load(epoch)

#**5. 토큰화를 하기 위해 필요한 코드이다.**
> ***원하는 epoch로 수정 가능!***

In [ ]:
from transformers import ElectraTokenizerFast

MAX_LEN = 256
TEST_BATCH_SIZE = 8
EPOCH = 72
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [ ]:
class ElectraDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index].strip()
        word_labels = self.data.label[index].split()

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        valid_token_list = []
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] == 0:
                continue
            valid_token_list.append(mapping)
        if len(valid_token_list) != len(word_labels):
            print(index, len(word_labels), len(valid_token_list), sentence)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        if len(labels) != 0:
            for idx, mapping in enumerate(encoding["offset_mapping"]):
                if mapping[0] == 0 and mapping[1] == 0:
                    continue
                tok = tokenizer.convert_ids_to_tokens(encoding['input_ids'][idx])
            
                # overwrite label
                if i == len(labels):
                    break
                encoded_labels[idx] = labels[i]
                i += 1
                
        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['label'] = torch.as_tensor(encoded_labels)
        
        return item
    def __len__(self):
        return self.len

In [ ]:
testing_set = ElectraDataset(df, tokenizer, MAX_LEN)

In [ ]:
test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

testing_loader = DataLoader(testing_set, **test_params)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
model.to(device)

In [ ]:
inputs = testing_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["label"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(0.0693, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 256, 152])

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

#**6. test 함수를 불러오는 섹션이다.**

In [ ]:
from sklearn.metrics import classification_report
def test():
    # put model in evaluation mode
    model.eval()
    
    test_loss, test_accuracy = 0, 0
    nb_test_examples, nb_test_steps = 0, 0
    test_preds, test_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['label'].to(device, dtype = torch.long)
            
            output = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = output[0]
            test_logits = output[1]
            
            test_loss += loss.item()

            nb_test_steps += 1
            nb_test_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = test_loss/nb_test_steps
                print(f"Test loss per 100 test steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = test_logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            test_labels.extend(labels)
            test_preds.extend(predictions)
            
            tmp_test_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            test_accuracy += tmp_test_accuracy

    labels = [ids_to_labels[id.item()] for id in test_labels]
    predictions = [ids_to_labels[id.item()] for id in test_preds]
    print(classification_report(labels, predictions))
    test_loss = test_loss / nb_test_steps
    test_accuracy = test_accuracy / nb_test_steps
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

#**TEST 결과**

In [ ]:
test()

#**(선택) predict 함수이다.**

In [ ]:
#예측하고 싶은 텍스트를 넣어주세요.
predict_text = '다진 마늘을 넣고 잘게 자른 김치를 섞어주세요.'

In [ ]:
def predict(sentence):
    result = []
    inputs = tokenizer(sentence,
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=MAX_LEN,
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    word_prediction = []
    token_prediction = []
    pred_dict = {}
    token_dict = {}
    for token_pred, word_idx in zip(wp_preds, inputs.word_ids()):
        if token_pred[0] not in ['[CLS]','[UNK]','[PAD]','[SEP]']:
            result.append([word_idx, token_pred[0], token_pred[1]]) # 
            #print(token_pred)
            if word_idx not in pred_dict:
                pred_dict[word_idx] = set()
            if word_idx not in token_dict:
                token_dict[word_idx] = ""
            pred_dict[word_idx].add(token_pred[1])
            token_dict[word_idx] += token_pred[0].replace("#", "")
                
    for token_pred, word_idx in zip(wp_preds, inputs.word_ids()):
        #only predictions on first word pieces are important
        if token_pred[0] not in ['[CLS]','[UNK]','[PAD]','[SEP]']:
            if token_pred[1] != 'O':
                token_prediction.append([word_idx, token_pred[0],token_pred[1]])
    
    #for i in range(len(token_prediction)):
    #    print(token_prediction[i], word_idx)
    #print()
    return result

In [ ]:
predict(predict_text)

[[0, '다진', 'CV_STATE'],
 [1, '마늘', 'CV_INGREDIENT'],
 [1, '##을', 'O'],
 [2, '넣', 'O'],
 [2, '##고', 'O'],
 [3, '잘', 'CV_STATE'],
 [3, '##게', 'CV_STATE'],
 [4, '자', 'CV_STATE'],
 [4, '##른', 'CV_STATE'],
 [5, '김치', 'CV_INGREDIENT'],
 [5, '##를', 'O'],
 [6, '섞', 'O'],
 [6, '##어', 'O'],
 [6, '##주', 'O'],
 [6, '##세요', 'O'],
 [7, '.', 'O']]